In [1]:
import os
import re
import yaml
import json
import ast
import praw
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

from textwrap import dedent
from reddit_helper import *    

from langchain.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI

from pydantic import BaseModel

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
api_file_path = 'api_keys.json'
with open(api_file_path, 'r') as file:
    api_keys = json.load(file)        
openai_gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])
openai_gpt4 = ChatOpenAI(model_name="gpt-4", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])        
llama31_8b = Ollama(model='llama3.1:8b',)
gemma2_9b = Ollama(model='gemma2:9b',)

In [42]:
cfg_file_path = 'casaai_config.yaml'
agent_cfg_path = 'agents.yaml'
task_cfg_path = 'tasks.yaml'

with open(cfg_file_path, 'r') as yaml_file:
    cfg = yaml.safe_load(yaml_file)
product_long = cfg.get('product_long_description', '')      
product_short = cfg.get('product_short_description', '')
domain = cfg.get('domain')
broad_keywords = cfg.get('broad_keywords', [])
output_format_1 = cfg.get('output_format_1')

with open(agent_cfg_path, 'r') as yaml_file:
    agent_cfg_data = yaml.safe_load(yaml_file)

with open(task_cfg_path, 'r') as yaml_file:
    task_cfg_data = yaml.safe_load(yaml_file)

In [ ]:
tip_text = "If you do your BEST WORK, I'll give you a $100 commission!"

In [50]:
def re_decision_string(input_string):

    pattern = r"'comment_id':\s*'([^']*)',\s*'decision':\s*(\d+)"

    # Use re.findall to extract the values
    matches = re.findall(pattern, input_string)

    if matches:
        comment_id, rank = matches[0]
    else:
        comment_id = "na"
        rank = 0

    justification = input_string.split("justification")[-1]
    
    return comment_id, decision, justification

In [51]:
def re_score_string(input_string):

    pattern = r"'comment_id':\s*'([^']*)',\s*'score':\s*(\d+)"

    # Use re.findall to extract the values
    matches = re.findall(pattern, input_string)

    if matches:
        comment_id, rank = matches[0]
    else:
        comment_id = "na"
        rank = 0

    justification = input_string.split("justification")[-1]
    
    return comment_id, score, justification

In [32]:
backstory = agent_cfg_data['content_filter_analyst']['backstory'] 
goal = agent_cfg_data['content_filter_analyst']['goal']
role = agent_cfg_data['content_filter_analyst']['role'] 
content_filter_analyst = Agent(
                            role=role,
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=False,
                            llm=gemma2_9b,
                            )

In [33]:
class DecisionOutput(BaseModel):
    comment_id: str
    decision: str
    justification: str

In [35]:
description = task_cfg_data['content_filter_task']['description']
expected_out = task_cfg_data['content_filter_task']['expected_out'] 
        
content_filter_task = Task(
                            description=description,
                            expected_output=expected_out,
                            output_json=DecisionOutput,
                            agent=content_filter_analyst,
                            )

In [36]:
response_creation_crew = Crew(
    agents=[content_filter_analyst,],
    tasks=[content_filter_task,],
    verbose=False,
)

2024-08-23 14:48:39,129 - 140220634834752 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [12]:
reddit_posts, reddit_post_ids = fetch_reddit_test()
condensed_reddit_data, unique_post_ids, unique_comment_ids = condense_data(reddit_posts, reddit_post_ids)

post_cnt: 0 + 10 = 10
comm_cnt: 0 + 560 = 560
cond_cnt: 570 = 10 + 560
Cross_ck: 10 = 10


In [13]:
comment_dict = {}
tot_data = 0
for idx1, item in enumerate(condensed_reddit_data):
    for idx2, data in enumerate(item):
        tot_data += 1
        c_id = data['comment_id']
        p_id = data['parent_id']
        text = data['text']
        comment_dict[c_id] = {'comment_id': c_id, 'parent_id': p_id, 'text': text}
    print(idx1, ':', idx2)
comment_dict_list = list(comment_dict.keys())
print(len(comment_dict), '=', tot_data, len(comment_dict_list))

0 : 169
1 : 108
2 : 31
3 : 57
4 : 0
5 : 33
6 : 67
7 : 60
8 : 1
9 : 34
570 = 570 570


In [14]:
def get_data_details(comm_id):
    done = 0
    data_details = []
    while done==0:
        if comm_id in comment_dict:
            details = comment_dict[comm_id]
            data_details.append(details)
            comm_id = details['parent_id']
        else:
            done = 1
    return data_details

In [ ]:
decision_lst = []
for idx, reddit_data_item in enumerate(condensed_reddit_data):
    comment_lst = []
    for idx2, comment_data in enumerate(reddit_data_item):        
        comm_id = comment_data['comment_id']  
        comment_text = comment_data['text']
        print(f'STARTING {idx} - {idx2} - {comm_id}')   
        input_dict = {"comment_id": comm_id,
                      "input_data": comment_text,
                      "product_long": product_long,
                      "product_short": product_short,
                      "domain": domain,
                      "broad_keywords": broad_keywords,
                      "output_format":output_format_1, 
                      "tip_text":tip_text}
        decision_result = response_creation_crew.kickoff(inputs=input_dict)
        try:
            json_out = ast.literal_eval(decision_result.json)
        except:
            print('Exception in json - trying re')
            _, d, j = re_decision_string(decision_result.raw)            
            json_out = {'comm_id': comm_id, 'decision': str(d), 'justification': str(j)}
        comment_lst.append(json_out)
    decision_lst.append(comment_lst)

In [59]:
analysis_lst = []
bad_list = []
tot_rel = 0
tot_comments = 0
bad_cnt = 0
relevant_json_list = []
discarded_json_list = []
for item in decision_lst:
    tot = 0
    rel = 0
    disc = 0    
    sep = "^"
    for comment in item:
        tot += 1
        tot_comments += 1
        try:
            comm_key = list(comment.keys())[0]
            comm_id  = comment[comm_key].strip()
            if comm_id in comment_dict_list:                 
                parent_id = comment_dict[comm_id]["parent_id"]
                if parent_id is None:
                    parent_id = 'NA'
                decision = comment['decision']
                justify  = comment["justification"]
                comment_text = comment_dict[comm_id]["text"]
                text_str = sep + comm_id + sep + parent_id + sep + decision + sep + justify + sep + comment_text + sep
                if comment['decision'] == 'relevant':
                    rel += 1
                    tot_rel += 1
                    relevant_json_list.append(comment)
                else:
                    disc += 1
                    discarded_json_list.append(comment)
                analysis_lst.append(text_str)            
            else: 
                print(f'Comment ID not found : {comm_id}')
                bad_cnt += 1
                bad_list.append(comm_id)
        except:
            print(f'comment key error : {comment}')
            bad_cnt += 1
    # print(f'tot : {tot}, rel : {rel}, discarded : {disc}, bad_cnt : {bad_cnt} analysis : {len(analysis_lst)}')
    # print(f'total relevant : {tot_rel}, total comments : {tot_comments}')
print(f'relevant : {len(relevant_json_list)} discarded : {len(discarded_json_list)} bad_cnt: {bad_cnt} total: {tot_comments}')

Comment ID not found : ggg8c8fu
Comment ID not found : ggggp581
Comment ID not found : ggggbtyv
Comment ID not found : ggevxe
Comment ID not found : [Comment ID]
Comment ID not found : ggfeoo
Comment ID not found : ggy9lj
Comment ID not found : gggglvit
Comment ID not found : ggg lx92
Comment ID not found : ggg3zh
Comment ID not found : kq75xk
Comment ID not found : krct1qh
Comment ID not found : kq754s
Comment ID not found : krcylty
Comment ID not found : your_comment_id
Comment ID not found : gtvj1c
relevant : 249 discarded : 305 bad_cnt: 16 total: 570


In [60]:
relevant_json_list[0]

{'comment_id': 'ggfh2zn',
 'decision': 'relevant',
 'justification': 'Seeks information about design choices.'}

In [39]:
with open('decision_result_v5_0823C.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in analysis_lst:
        file.write(item + "\n")  # Add a newline character after each string

print(f"List has been written")

List has been written


In [40]:
with open('decision_result_v5_0823C.txt', "r") as file:
    # Read all lines and strip the newline character from each line
    analysis_lst_loaded_back = [line.strip() for line in file.readlines()]
print(len(analysis_lst_loaded_back))

710


In [43]:
backstory = agent_cfg_data['content_scoring_analyst']['backstory'] 
goal = agent_cfg_data['content_scoring_analyst']['goal']
role = agent_cfg_data['content_scoring_analyst']['role'] 
content_scoring_analyst = Agent(
                            role=role,
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=False,
                            llm=gemma2_9b,
                            )

In [44]:
class ScoreOutput(BaseModel):
    comment_id: str
    score: float
    justification: str

In [45]:
description = task_cfg_data['content_scoring_task']['description']
expected_out = task_cfg_data['content_scoring_task']['expected_out'] 
        
content_scoring_task = Task(
                            description=description,
                            expected_output=expected_out,
                            output_json=ScoreOutput,
                            agent=content_filter_analyst,
                            )

In [46]:
score_creation_crew = Crew(
    agents=[content_scoring_analyst,],
    tasks=[content_scoring_task,],
    verbose=False,
)

2024-08-23 15:32:56,457 - 140220634834752 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [70]:
score_result_lst = []
for idx, comment_data in enumerate(relevant_json_list):     
    comm_id = comment_data['comment_id']        
    print(f'STARTING {idx} - {comm_id}')
    data_details = get_data_details(comm_id)      
    input_dict = {"comment_id": comm_id,
                  "input_data": data_details,
                  "product_long": product_long,
                  "product_short": product_short,
                  "domain": domain,
                  "tip_text":tip_text}
    scoring_result = score_creation_crew.kickoff(inputs=input_dict)
    try:
        json_out = ast.literal_eval(scoring_result.json)
    except:
        print('Exception in json - trying re')
        _, s, j = re_score_string(scoring_result.raw)            
        json_out = {'comm_id': comm_id, 'score': float(d), 'justification': str(j)}            
    score_result_lst.append(json_out)

STARTING 0 - ggfh2zn
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
STARTING 1 - ggewh8g
 Error parsing JSON: Expecting property name enclosed in double quotes: line 1 column 2 (char 1). Attempting to handle partial JSON.
 Pydantic validation error: 1 validation error for ScoreOutput
  Invalid JSON: key must be a string at line 1 column 2 [type=json_invalid, input_value="{'comment_id': 'ggewh8g'...AI's target audience.'}", input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/json_invalid. The JSON structure doesn't match the expected model. Attempting alternative conversion method.
STARTING 2 - ggfqz4a
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
STARTING 3 - ggf874w
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
STARTING 4 - ggg0jqv
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Att

In [71]:
score_result_lst[0], len(score_result_lst), len(relevant_json_list)

({'comment_id': 'ggfh2zn',
  'score': 9.5,
  'justification': 'Highly engaged user asking relevant design questions about the space.'},
 249,
 249)

In [86]:
thresh = 7.5
thresh_up = 0

for item in score_result_lst:
    if item['score'] >= thresh:
        thresh_up += 1
print(thresh_up, len(score_result_lst))

156 249


In [83]:
analysis_lst = []
bad_list = []
tot_comments = 0
bad_cnt = 0
good_json_list = []
sep = "^"
for comment in score_result_lst:
    tot_comments += 1
    try:
        comm_key = list(comment.keys())[0]
        comm_id  = comment[comm_key].strip()
        
        if comm_id in comment_dict_list:  
            parent_id = comment_dict[comm_id]["parent_id"]
            if parent_id is None:
                parent_id = 'NA'
            score = comment['score']
            justify  = comment["justification"]
            comment_text = comment_dict[comm_id]["text"]
            text_str = sep + comm_id + sep + parent_id + sep + str(score) + sep + justify + sep + comment_text + sep
            analysis_lst.append(text_str) 
            good_json_list.append(comm_id)
        else: 
            print(f'Comment ID not found : {comm_id}')
            bad_cnt += 1
            bad_list.append(comm_id)
    except:
        print(f'comment key error : {comment}')
        bad_cnt += 1
print(f'relevant : {len(good_json_list)} bad_cnt: {bad_cnt} total: {tot_comments}')

relevant : 249 bad_cnt: 0 total: 249


In [84]:
with open('score_result_v5_0823C.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in analysis_lst:
        file.write(item + "\n")  # Add a newline character after each string

print(f"List has been written")

List has been written


In [85]:
with open('score_result_v5_0823C.txt', "r") as file:
    # Read all lines and strip the newline character from each line
    analysis_lst_loaded_back = [line.strip() for line in file.readlines()]
print(len(analysis_lst_loaded_back))

349


In [64]:
# For each post-comment combo select the post-comments whose score >= 7

In [ ]:
# Then select a threshold like 20% or top 3 whichever is bigger of comments to craft response

In [ ]:
# Build response for these comments With Manager LLM + only 1 response writer + meta reviewer